In [ ]:
"""
Attempts at improving model predictions:

1. Could use bootstrapping due to dearth of data

2. Could use Ensemble methods(XGBoostRegressor, RandomForestRegressor)

3. Could use Blended Models/Stacking: Combine your best models. (Ridge + XGBoost + MLP) → feed their outputs into a final LinearRegression()
		from sklearn.ensemble import StackingRegressor
		estimators = [('ridge', ridge), ('mlp', mlp), ('xgb', xgb)]
		stack = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())

4. Adding in more variables(predictors):
		Customer age (in days) = Date of last purchase - first purchase
		Time since last purchase
		Purchase seasonality = Month of cohort or mode of purchase month
		Avg order value, purchase frequency per month
"""

In [35]:
# !pip install xgboost

In [2]:
import pandas as pd
import numpy as np
import kagglehub
import os
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from xgboost import XGBRegressor
from scipy import stats
from sklearn.base import clone

import warnings
warnings.filterwarnings('ignore')

### Setup

In [4]:
# https://www.kaggle.com/datasets/mashlyn/online-retail-ii-uci
path = kagglehub.dataset_download("mashlyn/online-retail-ii-uci")
print("Path to dataset files:", path)

file_path = os.path.join(path, 'online_retail_II.csv')
df = pd.read_csv(file_path)

# Loading and preprocessing the data
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Dropping rows with CustomerID as NULL
df = df.dropna(subset=['Customer ID'])

# Capping Outliers
df = df[(df['Quantity'] > 0) & (df['Quantity'] < df['Quantity'].quantile(0.99))]

# Defining Training cohorts: Considering Dec-2010, Jan-2011, Feb-2011 and Mar-2011 as training sample cohorts
training_cohorts = [
    ('2010-12-01', '2009-12-01', '2010-11-30', '2011-01-01', '2011-06-30'),
    ('2011-01-01', '2010-01-01', '2010-12-31', '2011-02-01', '2011-07-31'),
    ('2011-02-01', '2010-02-01', '2011-01-31', '2011-03-01', '2011-08-31'),
    ('2011-03-01', '2010-03-01', '2011-02-28', '2011-04-01', '2011-09-30')]

# Defining Out-Of-Time cohorts. Dataset ends ~Dec 9, 2011
holdout_cohorts = [
    ('2011-04-01', '2010-04-01', '2011-03-31', '2011-05-01', '2011-10-31'),
    ('2011-05-01', '2010-05-01', '2011-04-30', '2011-06-01', '2011-11-30'),
    ('2011-06-01', '2010-06-01', '2011-05-31', '2011-07-01', '2011-12-09')]

Path to dataset files: /Users/rohanojha/.cache/kagglehub/datasets/mashlyn/online-retail-ii-uci/versions/3


### Function to calculate the RFM variables and LTV

In [6]:
def calc_rfm_ltv(cohort_month, train_start, train_end, test_start, test_end, is_holdout=False, training_scaler=None):
    cohort_month = pd.to_datetime(cohort_month)
    train_start = pd.to_datetime(train_start)
    train_end = pd.to_datetime(train_end)
    test_start = pd.to_datetime(test_start)
    test_end = pd.to_datetime(test_end)
    
    cohort_customers = df[(df['InvoiceDate'] >= cohort_month) & 
                         (df['InvoiceDate'] < cohort_month + pd.offsets.MonthEnd(1))]['Customer ID'].unique()
    
    ###################### Filtering customers with sufficient lookback data ######################
    train = df[(df['InvoiceDate'] >= train_start) & (df['InvoiceDate'] <= train_end) & 
               (df['Customer ID'].isin(cohort_customers))]
    customer_transaction_counts = train.groupby('Customer ID')['Invoice'].nunique()
    valid_customers = customer_transaction_counts[customer_transaction_counts >= 2].index
    train = train[train['Customer ID'].isin(valid_customers)]
    cohort_customers = set(cohort_customers).intersection(valid_customers)

    ###################### Obtaining the country coalesced variable. This can be used in PCA to form clusters ######################
    country_map = {
        'United Kingdom': 'UK', 'EIRE': 'Europe', 'Germany': 'Europe', 'France': 'Europe', 'Belgium': 'Europe',
        'Portugal': 'Europe', 'Netherlands': 'Europe', 'Poland': 'Europe', 'Channel Islands': 'Europe',
        'Spain': 'Europe', 'Cyprus': 'Europe', 'Greece': 'Europe', 'Norway': 'Europe', 'Austria': 'Europe',
        'Sweden': 'Europe', 'Finland': 'Europe', 'Denmark': 'Europe', 'Italy': 'Europe', 'Switzerland': 'Europe',
        'Lithuania': 'Europe', 'Czech Republic': 'Europe', 'USA': 'North America', 'Canada': 'North America',
        'Australia': 'Asia-Pacific', 'Japan': 'Asia-Pacific', 'Singapore': 'Asia-Pacific', 'Thailand': 'Asia-Pacific',
        'Korea': 'Asia-Pacific', 'United Arab Emirates': 'Middle East & Africa', 'Nigeria': 'Middle East & Africa',
        'RSA': 'Middle East & Africa', 'Bahrain': 'Middle East & Africa', 'Israel': 'Middle East & Africa',
        'Saudi Arabia': 'Middle East & Africa', 'Lebanon': 'Middle East & Africa', 'Malta': 'Europe',
        'Iceland': 'Europe', 'Brazil': 'Others', 'West Indies': 'Others', 'Unspecified': 'Others',
        'European Community': 'Others'}
    train['Country_coalesced'] = train['Country'].map(country_map).fillna('Others')
    train['Country_encoded'] = train['Country_coalesced'].map(train['Country_coalesced'].value_counts(normalize=True))

    ###################### Computing the RFM predictors ######################
    rfm = train.groupby('Customer ID').agg({'InvoiceDate': [lambda x: (train_end - x.max()).days,
                                                            lambda x: (x.max() - x.min()).days,
                                                            lambda x: stats.mode(x.dt.month)[0]],
        'Invoice': 'nunique', 'Price': lambda x: (x * train.loc[x.index, 'Quantity']).sum(), 'Quantity': 'sum'})

    ###################### Flattening column names ######################
    rfm.columns = ['Recency', 'CustomerAge', 'Seasonality', 'Frequency', 'Monetary', 'TotalQuantity']
    
    ###################### Calculating Average Order Value ######################
    rfm['AvgOrderValue'] = rfm['Monetary'] / rfm['Frequency']
    
    ###################### Calculating Purchase Frequency per Month ######################
    training_months = (train_end - train_start).days / 30
    rfm['FreqPerMonth'] = rfm['Frequency'] / training_months
    
    ###################### Adding Country_encoded ######################
    rfm['Country_encoded'] = train.groupby('Customer ID')['Country_encoded'].first()
    
    ###################### Log-transforming Monetary, AvgOrderValue to reduce skewness ######################
    rfm['Monetary'] = np.log1p(rfm['Monetary'])
    rfm['AvgOrderValue'] = np.log1p(rfm['AvgOrderValue'])
    
    ###################### Computing LTV (predictor) ######################
    test = df[(df['InvoiceDate'] >= test_start) & (df['InvoiceDate'] <= test_end) & 
              (df['Customer ID'].isin(cohort_customers))]
    ltv = test.groupby('Customer ID')[['Quantity', 'Price']].apply(
        lambda x: (x['Quantity'] * x['Price']).sum(), include_groups=False).rename('LTV')
    rfm = rfm.join(ltv, how='left').fillna(0)
    rfm['LTV'] = np.log1p(rfm['LTV'])
    
    ###################### Applying Scaler Transformation if holdout sample ######################
    features = ['Recency', 'CustomerAge', 'Seasonality', 'Frequency', 'Monetary', 'TotalQuantity', 'AvgOrderValue', 'FreqPerMonth', 'Country_encoded']
    if is_holdout:
        rfm_scaled = training_scaler.transform(rfm[features])
        rfm[features] = rfm_scaled
        return rfm
    else:
        return rfm, features

### Processing Training Cohorts

In [8]:
scaler = StandardScaler()
ltv_scaler = StandardScaler()
df_rfm = pd.DataFrame()

for cohort_month, train_start, train_end, test_start, test_end in training_cohorts:
    rfm, features = calc_rfm_ltv(cohort_month, train_start, train_end, test_start, test_end)
    df_rfm = pd.concat([df_rfm, rfm.assign(Cohort=cohort_month)], ignore_index=True)

###################### Standardizing predictors and target ######################
df_rfm_scaled = scaler.fit_transform(df_rfm[features])
df_rfm[features] = df_rfm_scaled
df_rfm['LTV'] = ltv_scaler.fit_transform(df_rfm[['LTV']])

### Bootstrap Function

In [62]:
def bootstrap_sample(df, n_samples):
    sample_size = len(df)
    bootstrap_dfs = []
    for _ in range(n_samples):
        sample = df.sample(n=sample_size, replace=True)
        bootstrap_dfs.append(sample)
    return bootstrap_dfs

### Training Ensemble Models

In [10]:
def train_ensemble_models(df_rfm, features, n_bootstrap=100):
    X = df_rfm[features]
    y = df_rfm['LTV']

    rf_model = RandomForestRegressor(random_state=2)
    xgb_model = XGBRegressor(random_state=2)

    ###################### Hyperparameters ######################
    rf_param_grid = {'n_estimators': [100, 200], 'max_depth': [10, 20, None], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2]}
    xgb_param_grid = {'n_estimators': [100, 200], 'max_depth': [3, 6], 'learning_rate': [0.01, 0.1], 'reg_lambda': [1, 10]}

    ###################### Performing GridSearchCV to find the required hypermeters ######################
    rf_grid  = GridSearchCV(rf_model,  rf_param_grid,  cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    xgb_grid = GridSearchCV(xgb_model, xgb_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

    ###################### Fitting the models ######################
    rf_grid.fit(X, y)
    xgb_grid.fit(X, y)

    ###################### Choosing the Best Models ######################
    rf_model = rf_grid.best_estimator_
    xgb_model = xgb_grid.best_estimator_
    
    ###################### Bootstrapping training sample ######################
    # rf_predictions = []
    # xgb_predictions = []
    # bootstrap_samples = bootstrap_sample(df_rfm, n_samples=n_bootstrap)
    
    # for sample in bootstrap_samples:
    #     X_sample = sample[features]
    #     y_sample = sample['LTV']
        
        ###################### Training RandomForest ######################
        # rf_model.fit(X_sample, y_sample)
        # rf_pred = rf_model.predict(X)
        # rf_predictions.append(rf_pred)
        
        ###################### Training XGBoost ######################
        # xgb_model.fit(X_sample, y_sample)
        # xgb_pred = xgb_model.predict(X)
        # xgb_predictions.append(xgb_pred)
    
    ###################### Averaging predictions across the bootstrap samples ######################
    # rf_pred_avg = np.mean(rf_predictions, axis=0)
    # xgb_pred_avg = np.mean(xgb_predictions, axis=0)
    # final_pred = (rf_pred_avg + xgb_pred_avg) / 2
    
    ###################### Predictions ######################
    rf_pred = rf_model.predict(X)
    xgb_pred = xgb_model.predict(X)
    final_pred = (rf_pred + xgb_pred) / 2

    ###################### Training performance ######################
    print("\nTraining Set Performance:\n")
    print("RandomForest RMSE:", mean_squared_error(y, rf_pred, squared=False))
    print("RandomForest MAE:", mean_absolute_error(y, rf_pred))
    print("RandomForest R2:", r2_score(y, rf_pred))
    
    print("XGBoost RMSE:", mean_squared_error(y, xgb_pred, squared=False))
    print("XGBoost MAE:", mean_absolute_error(y, xgb_pred))
    print("XGBoost R2:", r2_score(y, xgb_pred))
    
    print("Stacked Ensemble RMSE:", mean_squared_error(y, final_pred, squared=False))
    print("Stacked Ensemble MAE:", mean_absolute_error(y, final_pred))
    print("Stacked Ensemble R2:", r2_score(y, final_pred))
    
    return rf_model, xgb_model, final_pred

###################### Invoking train_ensemble_model() ######################
rf_model, xgb_model, final_pred = train_ensemble_models(df_rfm, features)


Training Set Performance:

RandomForest RMSE: 0.5528353171185957
RandomForest MAE: 0.38524672649147207
RandomForest R2: 0.6943731121463816
XGBoost RMSE: 0.7332312018246462
XGBoost MAE: 0.5121995157236068
XGBoost R2: 0.462372004670785
Stacked Ensemble RMSE: 0.639289703041117
Stacked Ensemble MAE: 0.4466068282192547
Stacked Ensemble R2: 0.5913086755856004


In [24]:
xgb_importances = xgb_model.feature_importances_
importance_df = pd.DataFrame({'Feature': features, 'XGB_Importance': xgb_importances}).sort_values(by='XGB_Importance', ascending=False)
importance_df

,Feature,XGB_Importance
4,Monetary,0.561921
3,Frequency,0.141378
5,TotalQuantity,0.074810
8,Country_encoded,0.055909
0,Recency,0.051270
6,AvgOrderValue,0.040671
1,CustomerAge,0.038242
2,Seasonality,0.035800
7,FreqPerMonth,0.000000


### Processing Holdout Cohorts

In [158]:
df_holdout = pd.DataFrame()
for cohort_month, train_start, train_end, test_start, test_end in holdout_cohorts:
    rfm = calc_rfm_ltv(cohort_month, train_start, train_end, test_start, test_end, is_holdout=True, training_scaler=scaler)
    df_holdout = pd.concat([df_holdout, rfm.assign(Cohort=cohort_month)], ignore_index=True)

# Standardizing LTV for holdout
df_holdout['LTV'] = ltv_scaler.transform(df_holdout[['LTV']])

# Predicting on the holdout data
X_holdout = df_holdout[features]
rf_pred_holdout = rf_model.predict(X_holdout)
xgb_pred_holdout = xgb_model.predict(X_holdout)
final_pred_holdout = (rf_pred_holdout + xgb_pred_holdout) / 2

# Evaluating the holdout performance
print("\nHoldout Set Performance:\n")
print("RandomForest RMSE:", mean_squared_error(df_holdout['LTV'], rf_pred_holdout, squared=False))
print("RandomForest MAE:", mean_absolute_error(df_holdout['LTV'], rf_pred_holdout))
print("RandomForest R2:", r2_score(df_holdout['LTV'], rf_pred_holdout))
print("XGBoost RMSE:", mean_squared_error(df_holdout['LTV'], xgb_pred_holdout, squared=False))
print("XGBoost MAE:", mean_absolute_error(df_holdout['LTV'], xgb_pred_holdout))
print("XGBoost R2:", r2_score(df_holdout['LTV'], xgb_pred_holdout))
print("Stacked Ensemble RMSE:", mean_squared_error(df_holdout['LTV'], final_pred_holdout, squared=False))
print("Stacked Ensemble MAE:", mean_absolute_error(df_holdout['LTV'], final_pred_holdout))
print("Stacked Ensemble R2:", r2_score(df_holdout['LTV'], final_pred_holdout))


Holdout Set Performance:

RandomForest RMSE: 0.7668983148741471
RandomForest MAE: 0.5765329488601766
RandomForest R2: 0.26841321268188423
XGBoost RMSE: 0.7330994880677224
XGBoost MAE: 0.5436879629820903
XGBoost R2: 0.3314773610163301
Stacked Ensemble RMSE: 0.7439178444637154
Stacked Ensemble MAE: 0.5572196383729554
Stacked Ensemble R2: 0.3116009868911104
